In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df1 = pd.read_csv("records_final_series_ODI_bowlers_v1.csv")

In [4]:
df1.head()

,Player,Country,Mat,Overs,Maidens,Runs,Wickets,4w,Best,Avg,S/R,E/R,Series_Code,H/A,Decade_Index,Matches
0,McKenzie G D,Australia,1,7.4,0,22,2,0,22-Feb,11.00,30.00,2.20,193,Home,0,1
1,Mallett A A,Australia,1,8.0,1,34,3,0,Mar-34,11.33,21.33,3.19,193,Home,0,1
2,Stackpole K R,Australia,1,8.0,0,40,3,0,Mar-40,13.33,21.33,3.75,193,Home,0,1
3,Illingworth R,England,1,8.0,1,50,3,0,Mar-50,16.67,21.33,4.69,193,Away,0,1
4,Thomson A L,Australia,1,8.0,2,22,1,0,22-Jan,22.00,64.00,2.06,193,Home,0,1


In [5]:
#df1.sort_values(['Series_Code', 'Mat'], ascending=[True, False], inplace=True)

In [6]:
#df1

In [7]:
series = df1['Decade_Index'].values
number, series_unique = np.unique(series, return_inverse = True)

In [8]:
players = df1['Player'].values
player_names, player_indices = np.unique(players, return_index = True)
player_dict_temp_3 = {}
for i in df1.index:
    if df1["Player"][i] in player_dict_temp_3.keys():
        player_dict_temp_3[df1["Player"][i]] = player_dict_temp_3[df1["Player"][i]] + [i]
    else:
        player_dict_temp_3[df1["Player"][i]] = [i]

In [9]:
player_indices

array([  274,  3153,  1951, ...,  7410, 10370,  8655], dtype=int64)

In [11]:
series_codes = df1['Series_Code'].values
series_list, series_occurances, series_code = np.unique(series_codes, return_index=True, return_inverse = True)
type(series_list)

numpy.ndarray

In [10]:
series_to_number = {'Australia':0, 'Pakistan':1, 'New Zealand':2, 'West Indies':3, 'England':4, 'India' :5, 'Sri Lanka':6, 'Bangladesh':7, 'Zimbabwe':8, 'South Africa':9, 'Other':10} 
#opposition_country =[]
series_country = []
j = 0
for i in series_occurances:
    home = 0
    away = 0
    j = i
    s_code = df1['Series_Code'][i]
    while 1 != 0:
        #print(j)
        if df1['H/A'][j] == 'Home' and home == 0:
            #print(df['Country'][j])
            country = df1['Country'][j]
            home = home + 1
        if df1['Series_Code'][j]!=s_code or home ==1:
            break    
        j = j + 1 
    if country in series_to_number.keys():
        series_country.append(series_to_number[country])
    else:
        series_country.append(10)

In [11]:
#series_country

In [12]:
#len(series_country)

In [13]:
series_list = list(series_list)

In [14]:
trps = np.zeros(656)

for i in df1.index:
    avg = df1["Avg"][i]
    if not np.isnan(avg):
        s = df1["Series_Code"][i]
        index = np.where(series_list == s)[0]
        #print(index)
        trps[index] += df1["Runs"][i]

In [15]:
trps


array([ 1687.,  1136.,  1090.,   235.,  1089.,   216.,   462.,   642.,
         763.,   832.,   470.,   246.,   302.,  5106.,   303.,   595.,
         988.,   350.,   246.,   942.,   812.,   577.,   546.,   614.,
         682.,   781.,  4144.,   253.,  4780.,   743.,   637.,   689.,
        6353.,   433.,   648.,  1008.,  6293.,   871.,   818.,   604.,
         904.,   526.,   836.,  1300.,   599.,  1336.,  6087.,  1077.,
         212.,   967.,  1155.,   900., 10677.,   532.,  1570.,   711.,
        6206.,  1227.,   884.,   453.,   727.,  1103.,  1273.,   479.,
         440.,  1259.,  1933.,  1429.,  7325.,  3864.,  1521.,   982.,
        1342.,   968.,  1368.,   734.,  1688.,  5976.,  1246.,   580.,
        1510.,   673.,   833.,  1306.,   680.,   793.,  2365.,  1544.,
        1835.,  1691.,  2680.,  5133.,  2234.,  1060.,  2094.,   986.,
       11543.,   989.,  2601.,  5037.,   315.,  1319.,  1843.,  1805.,
         314.,   202.,   901.,  1848.,  1862.,  1416.,   330.,  5456.,
      

In [16]:
arpspc = np.zeros(11)
tmpc = np.zeros(11)
for i in range(len(trps)):
    tmpc[series_country[i]] += df1['Matches'][series_occurances[i]]
    arpspc[series_country[i]] += trps[i]

In [17]:
arpc=arpspc/tmpc

In [18]:
avr_average = np.average(arpc)

In [19]:
weights_arpc = np.average(arpc)/arpc

In [20]:
weights_arpc

array([0.94370681, 0.98110777, 1.00771905, 0.99912575, 0.99241729,
       0.93649992, 1.05536995, 1.0256387 , 1.02115162, 0.96084184,
       1.09907793])

In [21]:
weighted_runs_arr = []
for i in df1.index:
    #print(series_code[i])
    w = weights_arpc[series_country[series_code[i]]]
    weighted_runs_arr.append(df1['Runs'][i]*w)
#weighted_runs_arr

In [22]:
wtrps = np.zeros(656)
twps = np.zeros(656)
tops = np.zeros(656)
tbps = np.zeros(656)
balls = np.zeros(len(df1.index))
for i in df1.index:
    avg = df1["Avg"][i]
    if not np.isnan(avg):
        wtrps[series_code[i]] += weighted_runs_arr[i]
        twps[series_code[i]] += df1["Wickets"][i]
        balls[i] = math.floor(df1["Overs"][i])*6 + (df1["Overs"][i]- math.floor(df1["Overs"][i]))*10
        tbps[series_code[i]] += balls[i]
        
    tops[series_code[i]] += df1["Overs"][i]

In [23]:
bowl=np.zeros(len(df1.index))
for i in df1.index:
    bowl[i] = (wtrps[series_code[i]] - weighted_runs_arr[i])/(twps[series_code[i]]-df1["Wickets"][i])
    avg = df1["Avg"][i]
    bowl[i] = bowl[i]/avg
    if np.isnan(avg) or avg == 0:
        bowl[i] = 0

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


In [24]:
bowl1=np.zeros(len(df1.index))
for i in df1.index:
    bowl1[i] = (wtrps[series_code[i]] - weighted_runs_arr[i])/(tops[series_code[i]]-df1["Overs"][i])
    e_r = df1["E/R"][i]
    bowl1[i] = bowl1[i]/e_r
    if e_r == 0:
        bowl1[i] = 0
bowl1[:14]

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


array([1.36370116, 0.89551813, 0.7390475 , 0.56061911, 1.46953993,
       0.92743616, 0.57220758, 0.42319349, 0.723366  , 0.78647557,
       1.02538401, 0.97455203, 1.0081609 , 0.93556289])

In [25]:
bowl2=np.zeros(len(df1.index))
for i in df1.index:
    bowl2[i] = (tbps[series_code[i]] - balls[i])/(twps[series_code[i]]-df1["Wickets"][i])
    s_r = df1["S/R"][i]
    bowl2[i] = bowl2[i]/s_r
    if np.isnan(s_r) or s_r == 0:
        bowl2[i] = 0

In [26]:
def name(bowl, x):
    runs_metric = []
    player_dict_temp = {}


    for i in player_names:
        no_of_wickets=0
        spf_sum = 0
        no_series = 0
        no_balls = 0
        for j in player_dict_temp_3[i]:
            spf_sum += bowl[j]
            no_series += 1
            no_of_wickets += df1["Wickets"][j]
            #no_balls += balls[j]
        #if(no_balls > 4000):
        if(no_of_wickets > x):    
            #runs_metric_20.append(runs/no_of_matches)
            player_dict_temp[i] = spf_sum/no_series

    sum_values =sum(player_dict_temp.values())
    '''for k,v in s:
        sum_values += v
        #print(k,":",v)'''
    a = sum_values/len(player_dict_temp) 
    return player_dict_temp, a

In [27]:
def final_players(x):
    player_dict_temp_25, a_25 = name(bowl, x)
    player_dict_temp_35, a_35 = name(bowl1, x)
    player_dict_temp_45, a_45 = name(bowl2, x)
    avg_temp = [a_25, a_35, a_45]
    w_temp = np.average(avg_temp)/avg_temp

    player_dict_temp_55 = {}

    for i in player_dict_temp_45.keys():
        player_dict_temp_55[i] = (w_temp[0]*player_dict_temp_25[i]+w_temp[1]*player_dict_temp_35[i]+w_temp[2]*player_dict_temp_45[i])/np.sum(w_temp)

    s_55 = [(k,player_dict_temp_55[k]) for k in sorted(player_dict_temp_55, key=player_dict_temp_55.get, reverse = True)]
    return s_55

In [35]:
s_55 = final_players(50)
s_51 = final_players(80)

In [36]:
def print_country_lists(start_decade):
    players_list = {}
    for country in series_to_number.keys():
        #print(country)
        count=0
        players_list[country] = list()
        for k,v in s_51:
            first = player_indices[np.where(player_names == k)[0][0]]
            #print(first)
            #print(first,"->",df[' Decade_Index '][first])
            if df1['Country'][first] == country and df1['Decade_Index'][first] >= start_decade:
                player = str(k)+':'+str(v)
                players_list[country] = players_list[country]+[player]
                count += 1
            if count == 5:
                break
        if count < 5:
            players_list[country] = list()
            count = 0
            for k,v in s_55:
                first = player_indices[np.where(player_names == k)[0][0]]
            #print(first)
            #print(first,"->",df[' Decade_Index '][first])
                if df1['Country'][first] == country and df1['Decade_Index'][first] >= start_decade:
                    player = str(k)+':'+str(v)
                    players_list[country] = players_list[country]+[player]
                    count += 1
                if count == 5:
                    break
    for k in players_list.keys():
        if k == "Other":
            break
        print(k)
        for i in players_list[k]:
            print(i)
        print("\n")

In [37]:
print_country_lists(0)

Australia
Starc M A:1.2388290749405766
McGrath G D:1.1522008148158502
Lee B:1.146208211295317
McDermott C J:1.1277091912516441
Gillespie J N:1.1274653077834638


Pakistan
Saqlain Mushtaq:1.2071621381531552
Saeed Ajmal:1.1695551206953003
Shoaib Akhtar:1.156436966162778
Wasim Akram:1.1485713471496413
Waqar Younis:1.1067592236234771


New Zealand
Bond S E:1.3906297871768638
Boult T A:1.1765028320763427
Hadlee R J:1.1368787968044545
McClenaghan M J:1.0959435288650823
Mills K D:1.064429219922306


West Indies
Garner J:1.3061704626484367
Holding M A:1.2424226530862574
Patterson B P:1.0949245672119334
Ambrose C E L:1.0602952259660323
Narine S P:1.0460460901083466


England
Woakes C R:1.0622266411470598
Flintoff A:1.0598377194115727
Gough D:1.0246849457540568
Plunkett L E:1.0050320754042446
Anderson J M:0.9928827383675644


India
Mohammed Shami:1.10450568859448
Agarkar A B:1.079327742530633
Srinath J:1.0203012916836718
Ashwin R:1.01687582830828
Kapil Dev:1.0159956720620331


Sri Lanka
Muralith

In [38]:
print_country_lists(4)

Australia
Starc M A:1.2388290749405766
Lee B:1.146208211295317
Gillespie J N:1.1274653077834638
McKay C J:1.1222032495722754
Bracken N W:1.121290927554088


Pakistan
Saeed Ajmal:1.1695551206953003
Shoaib Akhtar:1.156436966162778
Mohammad Irfan:0.9917478587010401
Naved-ul-Hasan:0.9905020403259347
Mohammad Sami:0.9851674199654581


New Zealand
Bond S E:1.3906297871768638
Boult T A:1.1765028320763427
McClenaghan M J:1.0959435288650823
Mills K D:1.064429219922306
Oram J D P:1.0223405857280168


West Indies
Narine S P:1.0460460901083466
Taylor J E:1.035142409920444
Rampaul R:1.0075384251175263
Dillon M V:1.006450192573449
Bravo D J J:0.9842150700479286


England
Woakes C R:1.0622266411470598
Flintoff A:1.0598377194115727
Plunkett L E:1.0050320754042446
Anderson J M:0.9928827383675644
Finn S T:0.987434484853701


India
Mohammed Shami:1.10450568859448
Agarkar A B:1.079327742530633
Ashwin R:1.01687582830828
Khan Z:1.0012906650085416
Pathan I K:0.990086795460797


Sri Lanka
Mendis B A W:1.11045

In [32]:
def print_alltime_lists(start_decade):    
    for k,v in s_51:
        first = player_indices[np.where(player_names == k)[0][0]]
        #print(first,"->",df[' Decade_Index '][first])
        if df1['Decade_Index'][first] >= start_decade:
            print(k,":",v)

In [ ]:
last = player_dict_temp_3[k][-1]

In [33]:
print_alltime_lists(0)

Bond S E : 1.389097579056246
Garner J : 1.305133325499127
Holding M A : 1.2417925908587695
Starc M A : 1.2376191681540845
Donald A A : 1.2216394220226576
Saqlain Mushtaq : 1.2061866683910218
Boult T A : 1.1756741703575828
Pollock S M : 1.1740905002706719
Steyn D W : 1.1692471016571977
Saeed Ajmal : 1.1689634771634747
Imran Tahir : 1.1560239939938994
Shoaib Akhtar : 1.1554521863368241
McGrath G D : 1.1520650533675674
Nel A : 1.151831307125595
Wasim Akram : 1.147852779988122
Lee B : 1.1453442239059861
Ntini M : 1.14453607640886
Hadlee R J : 1.1364673427642138
Muralitharan M : 1.129878433621602
McDermott C J : 1.1270043750229475
Gillespie J N : 1.1269371505519625
Bracken N W : 1.1207504029055264
Mendis B A W : 1.109721769901392
Waqar Younis : 1.1058832721006693
Fleming D W : 1.1023002540912394
Lillee D K : 1.0940464942094417
Johnson M G : 1.0879537705745081
Warne S K : 1.086450603326279
Agarkar A B : 1.078587383147578
Imran Khan : 1.0777110369882252
Vaas W P U J C : 1.066788644842055
Mork

In [34]:
print_alltime_lists(4)

Bond S E : 1.389097579056246
Starc M A : 1.2376191681540845
Boult T A : 1.1756741703575828
Pollock S M : 1.1740905002706719
Steyn D W : 1.1692471016571977
Saeed Ajmal : 1.1689634771634747
Imran Tahir : 1.1560239939938994
Shoaib Akhtar : 1.1554521863368241
Nel A : 1.151831307125595
Lee B : 1.1453442239059861
Ntini M : 1.14453607640886
Gillespie J N : 1.1269371505519625
Bracken N W : 1.1207504029055264
Mendis B A W : 1.109721769901392
Johnson M G : 1.0879537705745081
Agarkar A B : 1.078587383147578
Morkel M : 1.0661697895769922
Mills K D : 1.0639692189492937
Woakes C R : 1.0618435429568256
Flintoff A : 1.0593348456731841
Taylor J E : 1.0346177478150886
Maharoof M F : 1.0277706553180066
Oram J D P : 1.022362510963004
Malinga S L : 1.0187898441110865
Ashwin R : 1.0166258099819074
Hogg G B : 1.0147482736236797
Zoysa D N T : 1.008519458690223
Rampaul R : 1.0070895304665528
Dillon M V : 1.0060735174172648
Khan Z : 1.0010397598732954
Anderson J M : 0.9925386310002567
Naved-ul-Hasan : 0.9898980

In [40]:
s_56 = final_players(5)
s_52 = final_players(20)

In [42]:
def print_country_lists_1(start_decade=8):
    players_list = {}
    for country in series_to_number.keys():
        #print(country)
        count=0
        players_list[country] = list()
        for k,v in s_52:
            last = player_dict_temp_3[k][-1]
            #print(first)
            #print(first,"->",df[' Decade_Index '][first])
            if df1['Country'][last] == country and df1['Decade_Index'][last] >= start_decade:
                player = str(k)+':'+str(v)
                players_list[country] = players_list[country]+[player]
                count += 1
            if count == 8:
                break
        if count < 8:
            players_list[country] = list()
            count = 0
            for k,v in s_56:
                last = player_dict_temp_3[k][-1]
            #print(first)
            #print(first,"->",df[' Decade_Index '][first])
                if df1['Country'][last] == country and df1['Decade_Index'][last] >= start_decade:
                    player = str(k)+':'+str(v)
                    players_list[country] = players_list[country]+[player]
                    count += 1
                if count == 8:
                    break
    for k in players_list.keys():
        if k == "Other":
            break
        print(k)
        for i in players_list[k]:
            print(i)
        print("\n")

In [43]:
print_country_lists_1()

Australia
Starc M A:1.2431613785938471
Hazlewood J R:1.0917035497786187
Cummins P J:1.0521387582105983
Coulter-Nile N M:0.9824595519625191
Faulkner J P:0.9714121732610436
Hastings J W:0.9659036649119732
Marsh M R:0.9472548328536828
Richardson K W:0.8751606514139835


Pakistan
Hasan Ali:1.466109422459048
Mohammad Amir:1.0845012670939982
Shadab Khan:1.0694963887423896
Mohammad Irfan:0.9926426782984397
Junaid Khan:0.9559151429320109
Umar Gul:0.9445113065753344
Wahab Riaz:0.93014534767647
Imad Wasim:0.9274449302561878


New Zealand
Henry M J:1.2888058917861416
Bennett H K:1.2445884224382424
Boult T A:1.1794444345935418
Santner M J:1.1125305210062377
Anderson C J:1.0637785645892877
Elliott G D:1.0435085204415562
Sodhi I S:0.9593584350717936
Bracewell D A J:0.959004320608666


West Indies
Narine S P:1.0452839278263695
Taylor J E:1.037080545526622
Bishoo D:0.9226049573491722
Miller N O:0.8667524404747831
Holder J O:0.847464740719066
Nurse A R:0.7914112267945215
Benn S J:0.763384359067206
Brat